# <center> <img src="../../img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> Computer Systems Engineering  </center>
---
### <center> Big Data Processing </center>
---
#### <center> **Autumn 2025** </center>
---

**Lab 03**: Data Cleaning and Transformation Pipeline

**Date**: September 19th 2025

**Student Name**: Diego Orozco Alvarado

**Professor**: Pablo Camarillo Ramirez

# Find the PySpark Installation

In [1]:
import findspark
findspark.init()

# Create SparkSession

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Examples on data sources (Files)") \
    .master("local[*]") \
    .config("spark.ui.port", "4040") \
    .getOrCreate()

sc = spark.sparkContext
sc.setLogLevel("ERROR")

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/09/19 15:05:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/09/19 15:06:11 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


## Define the schema

In [3]:
from pcamarillor.spark_utils import SparkUtils
airlines_schema_columns = [("index", "int"), 
     ("airline", "string"), 
     ("flight", "string"),
     ("source_city", "string"),
     ("departure_time", "string"),
     ("stops", "string"),
     ("arrival_time", "string"),
     ("destination_city", "string"),
     ("class", "string"),
     ("duration", "float"),
     ("days_left", "int"),
     ("price", "int")
     ]
airlines_schema = SparkUtils.generate_schema(airlines_schema_columns)
airlines_schema

StructType([StructField('index', IntegerType(), True), StructField('airline', StringType(), True), StructField('flight', StringType(), True), StructField('source_city', StringType(), True), StructField('departure_time', StringType(), True), StructField('stops', StringType(), True), StructField('arrival_time', StringType(), True), StructField('destination_city', StringType(), True), StructField('class', StringType(), True), StructField('duration', FloatType(), True), StructField('days_left', IntegerType(), True), StructField('price', IntegerType(), True)])

## Load CSV

In [4]:
df_airlines = spark.read \
                .option("header", "true") \
                .schema(airlines_schema) \
                .csv("/opt/spark/work-dir/data/airline/")

df_airlines.show(n=5)

+-----+--------+-------+-----------+--------------+-----+-------------+----------------+-------+--------+---------+-----+
|index| airline| flight|source_city|departure_time|stops| arrival_time|destination_city|  class|duration|days_left|price|
+-----+--------+-------+-----------+--------------+-----+-------------+----------------+-------+--------+---------+-----+
|    0|SpiceJet|SG-8709|      Delhi|       Evening| zero|        Night|          Mumbai|Economy|    2.17|        1| 5953|
|    1|SpiceJet|SG-8157|      Delhi| Early_Morning| zero|      Morning|          Mumbai|Economy|    2.33|        1| 5953|
|    2| AirAsia| I5-764|      Delhi| Early_Morning| zero|Early_Morning|          Mumbai|Economy|    2.17|        1| 5956|
|    3| Vistara| UK-995|      Delhi|       Morning| zero|    Afternoon|          Mumbai|Economy|    2.25|        1| 5955|
|    4| Vistara| UK-963|      Delhi|       Morning| zero|      Morning|          Mumbai|Economy|    2.33|        1| 5955|
+-----+--------+-------+

## Data Cleaning

In [5]:
from pyspark.sql.functions import trim, col, count, isnull, when
print(f"number of records before cleaning: {df_airlines.count()}")
# Get number of null values for each column before cleaning 
df_airlines.select([count(when(isnull(c[0]) | col(c[0]).isNull(), c[0])).alias(c[0]) for c in airlines_schema_columns]).show()

# Perform data cleaning with trim (column by column)
airlines_clean = df_airlines \
        .dropDuplicates(["index"]) \
        .withColumn("airline", trim("airline")) \
        .withColumn("source_city", trim("source_city")) \
        .withColumn("destination_city", trim("destination_city")) \
        .filter(col("price").isNotNull())


print(f"number of records after cleaning with trim: {airlines_clean.count()}")
airlines_clean.select([count(when(isnull(c[0]) | col(c[0]).isNull(), c[0])).alias(c[0]) for c in airlines_schema_columns]).show()
# Simply using dropna()
airlines_clean = df_airlines.dropna()

print(f"number of records after cleaning with dropna: {airlines_clean.count()}")
airlines_clean.select([count(when(isnull(c[0]) | col(c[0]).isNull(), c[0])).alias(c[0]) for c in airlines_schema_columns]).show()

number of records before cleaning: 300153


+-----+-------+------+-----------+--------------+-----+------------+----------------+-----+--------+---------+-----+
|index|airline|flight|source_city|departure_time|stops|arrival_time|destination_city|class|duration|days_left|price|
+-----+-------+------+-----------+--------------+-----+------------+----------------+-----+--------+---------+-----+
|    0|      0|     0|          0|             0|    0|           0|               0|    0|       0|        0|    0|
+-----+-------+------+-----------+--------------+-----+------------+----------------+-----+--------+---------+-----+



number of records after cleaning with trim: 300153


+-----+-------+------+-----------+--------------+-----+------------+----------------+-----+--------+---------+-----+
|index|airline|flight|source_city|departure_time|stops|arrival_time|destination_city|class|duration|days_left|price|
+-----+-------+------+-----------+--------------+-----+------------+----------------+-----+--------+---------+-----+
|    0|      0|     0|          0|             0|    0|           0|               0|    0|       0|        0|    0|
+-----+-------+------+-----------+--------------+-----+------------+----------------+-----+--------+---------+-----+



number of records after cleaning with dropna: 300153


+-----+-------+------+-----------+--------------+-----+------------+----------------+-----+--------+---------+-----+
|index|airline|flight|source_city|departure_time|stops|arrival_time|destination_city|class|duration|days_left|price|
+-----+-------+------+-----------+--------------+-----+------------+----------------+-----+--------+---------+-----+
|    0|      0|     0|          0|             0|    0|           0|               0|    0|       0|        0|    0|
+-----+-------+------+-----------+--------------+-----+------------+----------------+-----+--------+---------+-----+



## Create columns

In [6]:
from pyspark.sql.functions import when, lit
# Perform the 1st transformation

airlines_with_stops_numeric = airlines_clean.withColumn(
    "stops_numeric",
    when(col("stops") == "zero", lit(0))
    .when(col("stops") == "one", lit(1))
    .otherwise(lit(3))
)
airlines_with_stops_numeric.show(5)


+-----+--------+-------+-----------+--------------+-----+-------------+----------------+-------+--------+---------+-----+-------------+
|index| airline| flight|source_city|departure_time|stops| arrival_time|destination_city|  class|duration|days_left|price|stops_numeric|
+-----+--------+-------+-----------+--------------+-----+-------------+----------------+-------+--------+---------+-----+-------------+
|    0|SpiceJet|SG-8709|      Delhi|       Evening| zero|        Night|          Mumbai|Economy|    2.17|        1| 5953|            0|
|    1|SpiceJet|SG-8157|      Delhi| Early_Morning| zero|      Morning|          Mumbai|Economy|    2.33|        1| 5953|            0|
|    2| AirAsia| I5-764|      Delhi| Early_Morning| zero|Early_Morning|          Mumbai|Economy|    2.17|        1| 5956|            0|
|    3| Vistara| UK-995|      Delhi|       Morning| zero|    Afternoon|          Mumbai|Economy|    2.25|        1| 5955|            0|
|    4| Vistara| UK-963|      Delhi|       Morni

In [7]:
from pyspark.sql.functions import concat_ws, lit

airlines_with_route = airlines_with_stops_numeric.withColumn(
    "route",
    concat_ws(" -> ", col("source_city"), col("destination_city"))
)
airlines_with_route.show(3)


+-----+--------+-------+-----------+--------------+-----+-------------+----------------+-------+--------+---------+-----+-------------+---------------+
|index| airline| flight|source_city|departure_time|stops| arrival_time|destination_city|  class|duration|days_left|price|stops_numeric|          route|
+-----+--------+-------+-----------+--------------+-----+-------------+----------------+-------+--------+---------+-----+-------------+---------------+
|    0|SpiceJet|SG-8709|      Delhi|       Evening| zero|        Night|          Mumbai|Economy|    2.17|        1| 5953|            0|Delhi -> Mumbai|
|    1|SpiceJet|SG-8157|      Delhi| Early_Morning| zero|      Morning|          Mumbai|Economy|    2.33|        1| 5953|            0|Delhi -> Mumbai|
|    2| AirAsia| I5-764|      Delhi| Early_Morning| zero|Early_Morning|          Mumbai|Economy|    2.17|        1| 5956|            0|Delhi -> Mumbai|
+-----+--------+-------+-----------+--------------+-----+-------------+----------------+

In [8]:
from pyspark.sql.functions import when, col, lit

airlines_with_departure_and_arrival = airlines_with_route.withColumn(
    "departure_category",
    when(col("departure_time") == "Early_Morning", lit(0))
    .when(col("departure_time") == "Morning", lit(1))
    .when(col("departure_time") == "Afternoon", lit(2))
    .when(col("departure_time") == "Evening", lit(3))
    .when(col("departure_time") == "Night", lit(4))
    .otherwise(lit(5))
).withColumn(
    "arrival_category",
    when(col("arrival_time") == "Early_Morning", lit(0))
    .when(col("arrival_time") == "Morning", lit(1))
    .when(col("arrival_time") == "Afternoon", lit(2))
    .when(col("arrival_time") == "Evening", lit(3))
    .when(col("arrival_time") == "Night", lit(4))
    .otherwise(lit(5))
)

airlines_with_departure_and_arrival.show(10)




+-----+--------+-------+-----------+--------------+-----+-------------+----------------+-------+--------+---------+-----+-------------+---------------+------------------+----------------+
|index| airline| flight|source_city|departure_time|stops| arrival_time|destination_city|  class|duration|days_left|price|stops_numeric|          route|departure_category|arrival_category|
+-----+--------+-------+-----------+--------------+-----+-------------+----------------+-------+--------+---------+-----+-------------+---------------+------------------+----------------+
|    0|SpiceJet|SG-8709|      Delhi|       Evening| zero|        Night|          Mumbai|Economy|    2.17|        1| 5953|            0|Delhi -> Mumbai|                 3|               4|
|    1|SpiceJet|SG-8157|      Delhi| Early_Morning| zero|      Morning|          Mumbai|Economy|    2.33|        1| 5953|            0|Delhi -> Mumbai|                 0|               1|
|    2| AirAsia| I5-764|      Delhi| Early_Morning| zero|Ear

In [9]:
from pyspark.sql.functions import col, when, lit

airlines_with_is_expensive = airlines_with_departure_and_arrival.withColumn(
    "is_expensive",
    when(col("price") > 6000, lit(True)).otherwise(lit(False))
)
airlines_with_is_expensive.show(5)


+-----+--------+-------+-----------+--------------+-----+-------------+----------------+-------+--------+---------+-----+-------------+---------------+------------------+----------------+------------+
|index| airline| flight|source_city|departure_time|stops| arrival_time|destination_city|  class|duration|days_left|price|stops_numeric|          route|departure_category|arrival_category|is_expensive|
+-----+--------+-------+-----------+--------------+-----+-------------+----------------+-------+--------+---------+-----+-------------+---------------+------------------+----------------+------------+
|    0|SpiceJet|SG-8709|      Delhi|       Evening| zero|        Night|          Mumbai|Economy|    2.17|        1| 5953|            0|Delhi -> Mumbai|                 3|               4|       false|
|    1|SpiceJet|SG-8157|      Delhi| Early_Morning| zero|      Morning|          Mumbai|Economy|    2.33|        1| 5953|            0|Delhi -> Mumbai|                 0|               1|       fa

## aggregations

In [10]:
from pyspark.sql.functions import avg

avg_price_per_airline = airlines_with_is_expensive.groupBy("airline") \
    .agg(avg("price").alias("avg_price"))

avg_price_per_airline.show(3)


+---------+-----------------+
|  airline|        avg_price|
+---------+-----------------+
|   Indigo|5324.216303339517|
| SpiceJet|6179.278881367218|
|Air_India|23507.01911190229|
+---------+-----------------+
only showing top 3 rows


In [12]:
avg_duration_per_route = airlines_with_is_expensive.groupBy("route") \
    .agg(avg("duration").alias("avg_duration"))

avg_duration_per_route.show(5)


+--------------------+------------------+
|               route|      avg_duration|
+--------------------+------------------+
|    Delhi -> Chennai|12.433964745763944|
|  Hyderabad -> Delhi|10.829816602522587|
|   Mumbai -> Chennai|12.665900287564627|
|Hyderabad -> Kolkata|13.535322410033165|
| Hyderabad -> Mumbai|11.962923295795918|
+--------------------+------------------+
only showing top 5 rows


In [13]:
from pyspark.sql.functions import min, max

price_stats_per_airline = airlines_with_is_expensive.groupBy("airline") \
    .agg(
        min("price").alias("min_price"),
        max("price").alias("max_price")
    )

price_stats_per_airline.show()


+---------+---------+---------+
|  airline|min_price|max_price|
+---------+---------+---------+
|   Indigo|     1105|    31952|
| SpiceJet|     1106|    34158|
|Air_India|     1526|    90970|
|  AirAsia|     1105|    31917|
| GO_FIRST|     1105|    32803|
|  Vistara|     1714|   123071|
+---------+---------+---------+



In [18]:
from pyspark.sql.functions import count

count_by_departure_cat = airlines_with_is_expensive.groupBy("departure_category") \
    .agg(count("*").alias("flight_count"))

count_by_departure_cat.show()


+------------------+------------+
|departure_category|flight_count|
+------------------+------------+
|                 1|       71146|
|                 3|       65102|
|                 5|        1306|
|                 4|       48015|
|                 2|       47794|
|                 0|       66790|
+------------------+------------+



In [19]:
sc.stop()